In [ ]:
import numpy as np
import pandas as pd
import os
import h5py
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)

In [ ]:
total_path = r'Z:\zym\IS\control_1'
data_path = total_path + '\\' + 'plx'
#activate_path = total_path + '\\'+'activate'+'\\'+'plx'
#baseline_path = total_path + '\\'+'baseline'+'\\'+'plx'
PPR_path = total_path + '\\'+'PPR'+'\\'+'plx'
#PPR_path = total_path + '\\'+'PPR'+'\\'+'plx'
#filelist_a = os.listdir(activate_path)
#filelist_b = os.listdir(baseline_path)
filelist_d = os.listdir(data_path)
filelist_p = os.listdir(PPR_path)

plx_d=[]
for f in filelist_d:
    if f.endswith('.plx'):
        plx_d.append(f[:-4])
#plx_a=[]
#for f in filelist_a:
    #if f.endswith('.plx'):
        #plx_a.append(f[:-4])
#plx_b=[]
#for f in filelist_b:
    #if f.endswith('.plx'):
        #plx_b.append(f[:-4])
plx_p=[]
for f in filelist_p:
    if f.endswith('.plx'):
        plx_p.append(f[:-4])

In [ ]:
ptd_all = []
wave_data = []
unit_name = []
for idx,file_name in enumerate(plx_d):
    wave = pd.read_excel(os.path.join(data_path, file_name+'waveform.xlsx'),engine='openpyxl')
    wave = wave[wave.peak_trough_distance != 0]
    wave_data.extend(wave.values[:,0:3].tolist())
    unit_name.extend((wave.values[:,0]).tolist())
unit_name = list(dict.fromkeys(unit_name))
unit_name.sort()
for u in unit_name:
    ptd = []
    for w in wave_data:
        if w[0] == u:
            ptd.append(w[2])
    ptd_all.append(np.mean(ptd))
ptd_avg = np.array(ptd_all)
narrow_idx = np.argwhere((ptd_avg<0.4)|(ptd_avg==0.4)).flatten()
narrow_idx = narrow_idx.tolist()
broad_idx = np.argwhere(ptd_avg>0.4).flatten()
broad_idx = broad_idx.tolist()

In [ ]:
unit_narrow = []
for i in narrow_idx:
    unit_narrow.append(unit_name[i])
unit_broad = []
for i in broad_idx:
    unit_broad.append(unit_name[i])

In [ ]:
#wave_a = pd.read_excel(os.path.join(activate_path, plx_a[0]+'waveform.xlsx'),engine='openpyxl')
#wave_b = pd.read_excel(os.path.join(baseline_path, plx_b[0]+'waveform.xlsx'),engine='openpyxl')
wave_p = pd.read_excel(os.path.join(PPR_path, plx_p[0]+'waveform.xlsx'),engine='openpyxl')
#timestamp_a = h5py.File(os.path.join(activate_path, plx_a[0]+'timestamp.mat'),'r')
#timestamp_b = h5py.File(os.path.join(baseline_path, plx_b[0]+'timestamp.mat'),'r')
#kbd_a = timestamp_a['timestamp/kbd1'][:].flatten()

In [ ]:
delay = 5 #sec 
marker_a = np.arange(15,151,30)+kbd_a-delay
unit_list = list(wave_a.unit)
unit_name = []
gap = 15
for i in unit_list:
    name = 'timestamp/'+i
    unit_name.append(name)
    fig,ax = plt.subplots(2,1,figsize=(6,4))
    spike_a = timestamp_a[name][:].flatten()
    spk_list=[]
    for m in marker_a:
        start = m - gap
        stop = m + gap
        spk = spike_a[(spike_a>start) & (spike_a<stop)]-start
        spkl = spk.tolist()
        spk_list.append(spkl)
    spk_array = np.asarray(spk_list)
    ax[0].spines['top'].set_visible(False)
    ax[0].spines['right'].set_visible(False)
    ax[0].spines['bottom'].set_visible(False)
    ax[0].spines['left'].set_visible(False)
    ax[0].axes.xaxis.set_visible(False)
    ax[0].eventplot(spk_array, 'horizontal',linelengths=1,linewidths=0.2,color='slategray')
    ax[0].set_ylabel('#Trial',fontsize=14)
    ax[0].set_ylim(-0.5,4.5)
    ax[0].set_yticks(np.arange(0,4.1,1))
    ax[0].set_xlim(0,2*gap)
    ax[0].set_xticks(np.linspace(0,(2*gap/0.2-1)*0.2, len(np.arange(-gap, gap+0.1, gap/4))))
    ax[0].set_xticklabels(np.arange(-gap, gap+0.1, gap/4))
    ax[0].set_title(i)

    _hist = np.zeros((len(spk_array), len(np.arange(0, 2*gap, 0.2))-1))
    for row in range(len(spk_array)):
        spk_row = spk_array[row]
        distribution,_ = np.histogram(spk_row, np.arange(0, 2*gap, 0.2))
        _hist[row] = distribution
    hist = np.sum(_hist, 0)
    ax[1].bar((np.arange(0, 2*gap, 0.2)-0.1)[1:], hist, width = 0.2,color='slategray')
    title = '%s_PSTH_5trial.jpg'% i
    ax[1].spines['top'].set_visible(False)
    ax[1].spines['right'].set_visible(False)
    ax[1].spines['bottom'].set_linewidth('1')
    ax[1].spines['left'].set_linewidth('1')
    ax[1].set_ylabel('Counts',fontsize=14)
    ax[1].set_xlim(0,2*gap)
    ax[1].set_xticks(np.linspace(0,len(hist)*0.2, len(np.arange(-gap, gap+0.1, gap/4))))
    ax[1].set_xticklabels(np.arange(-gap, gap+0.1, gap/4))
    plt.savefig(os.path.join(activate_path, title),dpi=400)
    #plt.show()
    plt.close()

In [ ]:
unit_ppr = []
ppr_df = pd.DataFrame(columns=['day1','day2','day3'])
for file_name in plx_p:
    wave = pd.read_excel(os.path.join(PPR_path, file_name+'waveform.xlsx'),engine='openpyxl')
    wave = wave[wave.peak_trough_distance != 0]
    wave_data.extend(wave.values[:,0:3].tolist())
    unit_ppr.extend((wave.values[:,0]).tolist())
unit_ppr = list(dict.fromkeys(unit_ppr))
unit_ppr.sort()

In [ ]:
ppr_df = pd.DataFrame(columns=unit_ppr,index=['day1','day2','day3','cell_type'])
ppr_idv_df = pd.DataFrame(columns=unit_ppr,index=['day1','day2','day3','cell_type'])
kbd = np.arange(0,30,3.31)
gap = 1155/1000
#delay = 5 #sec
cal_gap = 500/1000
for idx,file_name in enumerate(plx_p):
    wave = pd.read_excel(os.path.join(PPR_path, file_name+'waveform.xlsx'),engine='openpyxl')
    timestamp = h5py.File(os.path.join(PPR_path, file_name+'timestamp.mat'),'r')
    kbd_p = timestamp['timestamp/kbd1'][:].flatten()
    kbd_p_list = kbd_p + kbd
    unit_list = list(wave.unit)
    unit_name = []
    for i in unit_list:
        name = 'timestamp/'+i
        unit_name.append(name)
        spike_p = timestamp[name][:].flatten()
        ppr_10trial = []
        for k_p in kbd_p_list:
            start1 = k_p + 5/1000
            stop1 = start1 + cal_gap
            start2 = start1 + gap
            stop2 = start2 + cal_gap
            #print([start1,stop1,start2,stop2])
            spk_array1 = spike_p[(spike_p>start1) & (spike_p<stop1)]
            spk_array2 = spike_p[(spike_p>start2) & (spike_p<stop2)]
            if len(spk_array1)/cal_gap != 0:
                ppr = (len(spk_array2)/cal_gap)/(len(spk_array1)/cal_gap)
            elif (len(spk_array1)==0) & (len(spk_array2)==0):
                ppr = 1
            else:
                ppr = 0
            ppr_10trial.append(ppr)
        ppr = np.mean(ppr_10trial)
        ppr_10trial = [float('{:.3f}'.format(i)) for i in ppr_10trial]
        ind = 'day%s'% (idx+1)
        ppr_df.loc[ind, i] = ppr
        ppr_idv_df.loc[ind, i] = ppr_10trial
        if i in unit_broad:
            ppr_df.loc['cell_type',i] = 'broad'
            ppr_idv_df.loc['cell_type',i] = 'broad'
        else:
            ppr_df.loc['cell_type',i] = 'narrow'
            ppr_idv_df.loc['cell_type',i] = 'narrow'
ppr_df.fillna(0,inplace=True)
ppr_df.to_excel(os.path.join(PPR_path,'ppr.xlsx'))
ppr_idv_df.to_excel(os.path.join(PPR_path,'ppr_10trial.xlsx'))

In [ ]:
ppr_df = pd.DataFrame(columns=unit_ppr,index=['day1','day2','day3','cell_type'])
gap = 1155/1000 
cal_gap = 500/1000
ppr_3day = []
pre_3day = []
post_3day = []
for idx,file_name in enumerate(plx_p):
    wave = pd.read_excel(os.path.join(PPR_path, file_name+'waveform.xlsx'),engine='openpyxl')
    timestamp = h5py.File(os.path.join(PPR_path, file_name+'timestamp.mat'),'r')
    kbd_p = timestamp['timestamp/kbd1'][:].flatten()
    start1 = kbd_p + 5/1000
    stop1 = start1 + cal_gap
    start2 = start1 + gap
    stop2 = start2 + cal_gap
    print([start1,stop1,start2,stop2])
    ppr_all = []
    unit_list = list(wave.unit)
    unit_name = []
    pre_all = []
    post_all = []
    
    for i in unit_list:
        name = 'timestamp/'+i
        unit_name.append(name)
        spike_p = timestamp[name][:].flatten()
        spk_array1 = spike_p[(spike_p>start1) & (spike_p<stop1)]
        spk_array2 = spike_p[(spike_p>start2) & (spike_p<stop2)]
        if len(spk_array1)/cal_gap != 0:
            ppr = (len(spk_array2)/cal_gap)/(len(spk_array1)/cal_gap)
        elif (len(spk_array1)==0) & (len(spk_array2)==0):
            ppr = 1
        else:
            ppr = 0
        ind = 'day%s'% (idx+1)
        ppr_df.loc[ind, i] = ppr
        if i in unit_broad:
            ppr_df.loc['cell_type',i] = 'broad'
        else:
            ppr_df.loc['cell_type',i] = 'narrow'
        ppr_all.append(ppr)
        pre_all.append(len(spk_array1)/cal_gap)
        post_all.append(len(spk_array2)/cal_gap)
    ppr_3day.append(ppr_all)
    post_3day.append(post_all)
    pre_3day.append(pre_all)
ppr_3day = np.array(ppr_3day)
post_3day = np.array(post_3day)
pre_3day = np.array(pre_3day)
pre_n_post = np.hstack((pre_3day,post_3day))

In [ ]:
#unit = [n+ '_pre' for n in unit_list] + [n+ '_post' for n in unit_list]
#df_ppr = pd.DataFrame(pre_n_post, columns=unit, index = ['day1','day2','day3'])
#df_ppr.to_excel(os.path.join(PPR_path,'ppr_data.xlsx'))
ppr_df.fillna(0,inplace=True)
ppr_df.to_excel(os.path.join(PPR_path,'ppr_1trial.xlsx'))

In [ ]:
for i in range(np.shape(ppr_3day)[1]):
    fig, ax = plt.subplots(figsize=(5,3))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth('1.2')
    ax.spines['left'].set_linewidth('1.2')
    x = np.arange(0,3,1)
    plt.plot(x, ppr_3day[:,i])
    plt.scatter(x, ppr_3day[:,i],c='slategray',edgecolors='k')
    plt.ylabel('PPR',fontsize=14)
    plt.xticks(np.arange(0,3,1),['day1','day2','day3'],fontsize=12)         
    plt.title(unit_list[i])
    plt.savefig(os.path.join(PPR_path,'%s_PPR.jpg'%unit_list[i]),dpi=300,bbox_inches = 'tight')
    plt.close()

In [ ]:
df_total = pd.DataFrame(columns=['unit','peak_trough_distance','type','bl_fr','fr_ratio'])
gap = 15
m_a = kbd_a-5
marker_b = 15
unit_list = list(wave_b.unit)
unit_name = []
for idx,i in enumerate(unit_list):
    fig,ax = plt.subplots(2,1,figsize=(6,4))
    name = 'timestamp/'+i
    unit_name.append(name)
    spike_b = timestamp_b[name][:].flatten()
    start_b = marker_b
    stop_b = marker_b + gap
    spk_array_b = spike_b[(spike_b>start_b) & (spike_b<stop_b)]-start_b
    fr_b = len(spk_array_b)/gap
    spike_a = timestamp_a[name][:].flatten()
    start_a = m_a
    stop_a = m_a + gap
    spk_array_a = spike_a[(spike_a>start_a) & (spike_a<stop_a)]-start_a + gap
    fr_a = len(spk_array_a)/gap
    spk_array = np.concatenate((spk_array_b,spk_array_a),axis=0)
    fr_ratio = fr_a/fr_b
    ax[0].spines['top'].set_visible(False)
    ax[0].spines['right'].set_visible(False)
    ax[0].spines['bottom'].set_visible(False)
    ax[0].spines['left'].set_visible(False)
    ax[0].axes.xaxis.set_visible(False)
    ax[0].eventplot(spk_array, 'horizontal',linelengths=1,linewidths=0.2,color='slategray')
    #ax[0].set_ylabel('#Trial',fontsize=14)
    #ax[0].set_ylim(-0.5,0.5)
    ax[0].set_yticklabels(['1','',''])
    ax[0].set_xlim(0,2*gap)
    ax[0].set_xticks(np.linspace(0,(2*gap/0.2-1)*0.2, len(np.arange(-gap, gap+0.1, gap/4))))
    ax[0].set_xticklabels(np.arange(-gap, gap+0.1, gap/4))
    ax[0].set_title(i)

    _hist = np.zeros((len(spk_array), len(np.arange(0, 2*gap, 0.2))-1))
    for row in range(len(spk_array)):
        spk_row = spk_array[row]
        distribution,_ = np.histogram(spk_row, np.arange(0, 2*gap, 0.2))
        _hist[row] = distribution
    hist = np.sum(_hist, 0)
    ax[1].bar((np.arange(0, 2*gap, 0.2)-0.1)[1:], hist, width = 0.2,color='slategray')
    title = '%s_PSTH.jpg'% i
    ax[1].spines['top'].set_visible(False)
    ax[1].spines['right'].set_visible(False)
    ax[1].spines['bottom'].set_linewidth('1')
    ax[1].spines['left'].set_linewidth('1')
    ax[1].set_ylabel('Counts',fontsize=14)
    ax[1].set_xlim(0,2*gap)
    ax[1].set_xticks(np.linspace(0,len(hist)*0.2, len(np.arange(-gap, gap+0.1, gap/4))))
    ax[1].set_xticklabels(np.arange(-gap, gap+0.1, gap/4))
    plt.savefig(os.path.join(baseline_path, title),dpi=400)
    plt.close()
    fig, ax = plt.subplots(figsize=(1.5,2.5))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth('1.2')
    ax.spines['left'].set_linewidth('1.2')
    plt.bar([1,2],[fr_b, fr_a], color='steelblue')
    plt.xticks(np.linspace(0,3,4),('','bl','chr2',''),fontsize=12)
    #plt.text(3.5,1,' n=%d \ncell=%d'%(mouse_num, cell_n))
    plt.xlim(0, 3)
    plt.ylabel('firing rate(Hz)',fontsize=14)
    title = '%s_FR.jpg'% i
    plt.savefig(os.path.join(baseline_path, title),dpi=300, bbox_inches = 'tight')
    plt.close()
    df_total.loc[idx] = [unit_list[idx], ptd_avg[idx],units[idx],fr_b, fr_ratio]
df_total.to_excel(os.path.join(baseline_path,'total.xlsx'))